In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import duckdb

# For plotting
from sklearn.decomposition import PCA

This is the nrg_d dataset

In [ ]:
dataset = pd.read_csv('/Users/couch/Eurostat_data/nrg_d/estat_nrg_d_hhq.tsv.gz', sep='\t', compression='gzip')
print(dataset.columns)

Index(['freq,nrg_bal,siec,unit,geo\TIME_PERIOD', '2010 ', '2011 ', '2012 ',
       '2013 ', '2014 ', '2015 ', '2016 ', '2017 ', '2018 ', '2019 ', '2020 ',
       '2021 ', '2022 ', '2023 '],
      dtype='object')


In [3]:
# data cleaning
dataset[['freq', 'nrg_bal', 'siec', 'unit', 'geo',]] = dataset['freq,nrg_bal,siec,unit,geo\TIME_PERIOD'].str.split(',', expand=True)
dataset = dataset.drop(columns=['freq,nrg_bal,siec,unit,geo\TIME_PERIOD'])
dataset.columns = dataset.columns.str.strip()
dataset.replace({': ': 0, ': m': np.nan}, inplace=True)
dataset[['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
               '2020', '2021', '2022', '2023']] = dataset[['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
               '2020', '2021', '2022', '2023']].apply(pd.to_numeric, errors='coerce')
dataset.columns

<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:3: SyntaxWarning: invalid escape sequence '\T'
<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:3: SyntaxWarning: invalid escape sequence '\T'
/var/folders/t3/pjw083nx7_b0p78lsvshw6yc0000gn/T/ipykernel_75743/2975463527.py:2: SyntaxWarning: invalid escape sequence '\T'
  dataset[['freq', 'nrg_bal', 'siec', 'unit', 'geo',]] = dataset['freq,nrg_bal,siec,unit,geo\TIME_PERIOD'].str.split(',', expand=True)
/var/folders/t3/pjw083nx7_b0p78lsvshw6yc0000gn/T/ipykernel_75743/2975463527.py:3: SyntaxWarning: invalid escape sequence '\T'
  dataset = dataset.drop(columns=['freq,nrg_bal,siec,unit,geo\TIME_PERIOD'])


Index(['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020', '2021', '2022', '2023', 'freq', 'nrg_bal', 'siec',
       'unit', 'geo'],
      dtype='object')

In [4]:
dataset.head()

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,freq,nrg_bal,siec,unit,geo
0,2997.450,2936.634,3065.799,4066.970,3500.001,3240.375,2990.930,2934.535,3146.976,3120.242,3469.079,3504.652,3358.820,3284.260,A,FC_OTH_HH_E,E7000,GWH,AL
1,17657.493,17273.639,17541.315,17649.164,16815.552,17215.299,17838.837,17998.218,17942.604,18456.640,19485.982,21317.241,21282.946,21922.902,A,FC_OTH_HH_E,E7000,GWH,AT
2,0.000,0.000,0.000,0.000,4604.000,4725.000,4734.000,4755.000,4685.000,4725.999,4795.000,4912.000,4929.000,5000.000,A,FC_OTH_HH_E,E7000,GWH,BA
3,0.000,0.000,0.000,0.000,0.000,0.000,18849.100,18537.100,18508.100,18357.200,17509.400,17586.900,16220.000,15884.300,A,FC_OTH_HH_E,E7000,GWH,BE
4,10559.000,10912.000,10838.000,10537.000,10590.000,10644.000,10733.000,11137.538,10963.025,10843.892,11356.181,11953.294,11969.685,12311.736,A,FC_OTH_HH_E,E7000,GWH,BG


In [5]:
dataset['freq'].unique()

array(['A'], dtype=object)

In [8]:
dataset['nrg_bal'].unique()

array(['FC_OTH_HH_E', 'FC_OTH_HH_E_CK', 'FC_OTH_HH_E_LE',
       'FC_OTH_HH_E_OE', 'FC_OTH_HH_E_SC', 'FC_OTH_HH_E_SH',
       'FC_OTH_HH_E_WH'], dtype=object)

This is nrg_d for industry based on caloric values

In [11]:
data_2 = pd.read_csv('/Users/couch/Eurostat_data/nrg_d/estat_nrg_d_indcv.tsv.gz', sep='\t', compression='gzip')
data_2.head()

,"freq,unit,siec,calval,geo\TIME_PERIOD",2017,2018,2019,2020,2021,2022,2023
0,"A,MJ_T_NCV,C0000X0350-0370,NCV_AVG,AL",:,:,:,:,25208.356,25140.805,:
1,"A,MJ_T_NCV,C0000X0350-0370,NCV_AVG,AT",:,:,:,27791.671,27585.598,27840.987,27458.397
2,"A,MJ_T_NCV,C0000X0350-0370,NCV_AVG,BE",:,:,:,19995.899,20150.621,20185.649,:
3,"A,MJ_T_NCV,C0000X0350-0370,NCV_AVG,BG",:,:,:,25730.389,25151.103,24009.095,24369.700
4,"A,MJ_T_NCV,C0000X0350-0370,NCV_AVG,CY",:,:,:,26135.000,25280.000,24302.000,:


In [12]:
data_2.columns

Index(['freq,unit,siec,calval,geo\TIME_PERIOD', '2017 ', '2018 ', '2019 ',
       '2020 ', '2021 ', '2022 ', '2023 '],
      dtype='object')

In [14]:
# data cleaning
data_2[['freq', 'unit', 'siec', 'calval', 'geo']] = data_2['freq,unit,siec,calval,geo\TIME_PERIOD'].str.split(',', expand=True)
data_2 = data_2.drop(columns=['freq,unit,siec,calval,geo\TIME_PERIOD'])
data_2.columns = data_2.columns.str.strip()
data_2.replace({': ': 0, ': m': np.nan}, inplace=True)
data_2[['2017', '2018', '2019', '2020', '2021', '2022', '2023']] = data_2[['2017', '2018', '2019',
               '2020', '2021', '2022', '2023']].apply(pd.to_numeric, errors='coerce')
data_2.columns

<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:3: SyntaxWarning: invalid escape sequence '\T'
<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:3: SyntaxWarning: invalid escape sequence '\T'
/var/folders/t3/pjw083nx7_b0p78lsvshw6yc0000gn/T/ipykernel_75743/4021475061.py:2: SyntaxWarning: invalid escape sequence '\T'
  data_2[['freq', 'unit', 'siec', 'calval', 'geo']] = data_2['freq,unit,siec,calval,geo\TIME_PERIOD'].str.split(',', expand=True)
/var/folders/t3/pjw083nx7_b0p78lsvshw6yc0000gn/T/ipykernel_75743/4021475061.py:3: SyntaxWarning: invalid escape sequence '\T'
  data_2 = data_2.drop(columns=['freq,unit,siec,calval,geo\TIME_PERIOD'])


Index(['2017', '2018', '2019', '2020', '2021', '2022', '2023', 'freq', 'unit',
       'siec', 'calval', 'geo'],
      dtype='object')

In [15]:
data_2.head()

,2017,2018,2019,2020,2021,2022,2023,freq,unit,siec,calval,geo
0,0.0,0.0,0.0,0.000,25208.356,25140.805,0.000,A,MJ_T_NCV,C0000X0350-0370,NCV_AVG,AL
1,0.0,0.0,0.0,27791.671,27585.598,27840.987,27458.397,A,MJ_T_NCV,C0000X0350-0370,NCV_AVG,AT
2,0.0,0.0,0.0,19995.899,20150.621,20185.649,0.000,A,MJ_T_NCV,C0000X0350-0370,NCV_AVG,BE
3,0.0,0.0,0.0,25730.389,25151.103,24009.095,24369.700,A,MJ_T_NCV,C0000X0350-0370,NCV_AVG,BG
4,0.0,0.0,0.0,26135.000,25280.000,24302.000,0.000,A,MJ_T_NCV,C0000X0350-0370,NCV_AVG,CY


nrg_ind dataset

In [16]:
dat = pd.read_csv('/Users/couch/Eurostat_data/estat_nrg_ind_342m.tsv.gz', sep='\t', compression='gzip')
dat.head()

,"freq,unit,product,indic_nrg,geo\TIME_PERIOD",2008-01,2008-02,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,"M,GWH,6000,B_190300,AT",2503,2284,2162,1764,1287,823,770,827,1223,...,2592,1750,1589,1282,1833,1718,2225,2861,1986,2685
1,"M,GWH,6000,B_190300,BE",1588,1569,2131,1886,1603,1469,1034,1495,1248,...,1211,1048,1269,1415,813,551,335,642,1032,1323
2,"M,GWH,6000,B_190300,BG",467,544,318,141,118,213,227,284,191,...,275,274,352,372,197,151,148,170,267,379
3,"M,GWH,6000,B_190300,CY",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"M,GWH,6000,B_190300,CZ",1065,865,708,569,507,545,751,592,483,...,1035,884,632,788,1346,740,847,853,663,1128


In [17]:
dat.columns

Index(['freq,unit,product,indic_nrg,geo\TIME_PERIOD', '2008-01 ', '2008-02 ',
       '2008-03 ', '2008-04 ', '2008-05 ', '2008-06 ', '2008-07 ', '2008-08 ',
       '2008-09 ',
       ...
       '2019-03 ', '2019-04 ', '2019-05 ', '2019-06 ', '2019-07 ', '2019-08 ',
       '2019-09 ', '2019-10 ', '2019-11 ', '2019-12 '],
      dtype='object', length=145)